In [1]:
import numpy as np
import pandas as pd
import glob
import zipfile

from datetime import datetime
import geopandas as gpd
%matplotlib inline

In [2]:
import os
import psutil
process = psutil.Process(os.getpid())
print(process.memory_info().rss/1e6)

185.634816


# 1. readin data

In [4]:
all_data_annual = pd.read_csv("0_csv_annual/HydroLakes_areas1_after_voro.csv.zip")
display(all_data_annual.head(3))

,system:index,Hylak_id,nodata,notwater,perwater,seawater,.geo
0,00040000000000009dd0,1274580,"[34637.177337646484, 0, 0, 427.5446472167969, ...","[40197.857849121094, 2138.1473388671875, 6841....","[528956.8090209961, 564878.361328125, 556753.1...","[1282.6599731445312, 38057.995513916016, 41480...",NaN
1,00040000000000009dd3,1350857,"[40344.039489746094, 43208.73025189568, 0, 0, ...","[144864.84893798828, 128971.92370605469, 12102...","[305637.1452026367, 320918.58673095703, 328253...","[4686.381985294118, 2433.1749267578125, 46252....",NaN
2,00040000000000009dd8,1352308,"[134098.7028157552, 189981.51125416474, 4310.1...","[129497.31998721854, 128412.71898456648, 15779...","[242089.3590451708, 223616.1765500536, 306637....","[63416.76413741767, 27091.739196777344, 100359...",NaN


# 2. define functions

In [5]:
def rm_outlier(mdata):
    
    cons_year = 1980
    df0 = mdata[mdata['month'] >= 0]
    df1 = df0[df0['water_enh'] > 0].copy()
    df2 = df0[df0['water_noenh'] > 0].copy()
    ##nname = fname.split('/')[1].split('.')[0]
    if len(df1.index) >= 7:
        
        ################## remove area outliers ##################
        water_enh_int = df1['water_enh'].values
        N = 7 # moving window
        weight = np.arange(1,N,1)[int(N/2):]
        weight = np.concatenate((weight, np.ones(len(water_enh_int)-N+1)*7, np.flip(weight, 0)))/7
        
        nol = 2
        limit = 50
        while nol > 0 and limit > 0:
            ########## remove outlier ###################
            wt_ma = np.convolve(water_enh_int, np.ones((N,))/N, mode='same')
            
            bias = water_enh_int - wt_ma/weight
            b_avg = np.average(bias)
            b_sd = np.std(bias)
            
            outlier = [0 if (x >= b_avg+3*b_sd or x <= b_avg-3*b_sd) else 1 for x in bias]
            
            n_mth = [x for x,o in zip(df1['month']*outlier, outlier) if o>0]
            n_enh = [x for x,o in zip(water_enh_int*outlier, outlier) if o>0]
            
            if len(n_mth) == 0:
                ndf = pd.DataFrame([])
                return ndf
            n_water_enh_int = np.interp(df1['month'], n_mth, n_enh)
            
            ############### add back normal ####################
            n_bias = water_enh_int - np.convolve(n_water_enh_int, np.ones((N,))/N, mode='same')/weight
            outlier = [0 if (x >= b_avg+3*b_sd or x <= b_avg-3*b_sd) else 1 for x in n_bias]
            
            n_mth = [x for x in df1['month']*outlier if x>0 ]
            n_enh = [x for x in water_enh_int*outlier if x>0 ]
            water_enh_int = np.interp(df1['month'], n_mth, n_enh)
            
            inreduce = nol - (len(outlier) - np.sum(outlier))
            nol = len(outlier) - np.sum(outlier)
            limit = limit - 1
            
            if inreduce == 0 and nol <=2:
                break
        
        ndf = pd.DataFrame({'1month':df0['month'], '2cloud_pct': df0['cloud_pct'], 
                            '3water_enh': np.interp(df0['month'], df1['month'], water_enh_int), 
                            '4water_noenh': np.interp(df0['month'], df2['month'], df2['water_noenh']), 
                            '5water_enh_raw': df0['water_enh'], '6water_noenh_raw': df0['water_noenh']})
        ndf['7intp_flag'] = [0 if c==d else 1 for c,d in 
                             zip(ndf['3water_enh'].values, ndf['5water_enh_raw'].values)]
        
        ndf['1month'] = pd.date_range(start='3/1/1984', periods=len(ndf), freq='MS')
        ndf[ndf['1month'] <= datetime.strptime(str(cons_year) + '-6-15', "%Y-%m-%d")] = np.nan
        ndf['1month'] = pd.date_range(start='3/1/1984', periods=len(ndf), freq='MS')
    else:
        ndf = pd.DataFrame([])
    
    return ndf

def final_area(a_mon, cld_yr, a_perm, a_sea, a_recons, intp_num):
    
    ## if more than 11 monthly values of these year are interpolated,
    ## then use reconstructed (previous: just use annual average value)
        
    if intp_num >= 11:
        area = a_recons
    else:
        area = a_mon
    
    ## if monthly value is larger than a_perm+a_sea in clear annual image,
    ## then limit the area. similarly for lower bound
    #if cld_yr<=0.05:
    area = a_perm+a_sea if area>a_perm+a_sea else area
    area = a_perm if area<a_perm else area
    
    return area

def yr_constrain(df):
    
    intp_num = df[['year', '7intp_flag']].groupby('year').sum()
    intp_num.at[1984, '7intp_flag'] = intp_num.at[1984, '7intp_flag'] + 2
    intp_num = intp_num.reset_index().rename({'7intp_flag': 'intp_num'}, axis=1)

    ndf = df.merge(intp_num, on='year', how='left')
    
    ndf['8water_area'] = ndf.apply(lambda x: final_area(x['3water_enh'], x['cloud_pct_yr'],
                                                        x['perwater_new'], x['seawater_new'],
                                                        x['recons_mthly'], x['intp_num']), axis=1)
    return ndf

def str2list(mstr):
    return mstr.replace('[','').replace(']','').replace(' ','').split(',')

In [7]:
large_hylak_ids = [1,2,3,4,8,10,11,12,17,18,20,21,22,23,24,25,26,28,31,35,47,50,58,59,63,90,91]

season_data = np.load('/mnt/data/Evap_CMIP6/0_JRC_areas/0_seasonality/seasonal_frac.npy')
## id from 2 to 1427688

def row2df(row):
    
    hylak_id = row['Hylak_id']
    
    ##########################################################################
    df = pd.DataFrame([])
    df['month_fmt'] = pd.date_range('1984-3-1','2020-12-31',freq='MS')
    df['month'] = np.arange(1, len(df)+1)
    df['water_enh'] = [float(c) for c in str2list(row['water_enh'])]
    df['water_noenh'] = [float(c) for c in str2list(row['water_noenh'])]
    df['cloud_pct'] = [float(c) for c in str2list(row['cloud_pct'])]
    
    ndf = rm_outlier(df)
    if len(ndf) == 0:
        return pd.DataFrame([])
    
    ndf['hylak_id'] = hylak_id
    
    ##########################################################################
    nodata   = np.array(str2list(row['nodata'])).astype(float)/1e6
    notwater = np.array(str2list(row['notwater'])).astype(float)/1e6
    perwater = np.array(str2list(row['perwater'])).astype(float)/1e6
    seawater = np.array(str2list(row['seawater'])).astype(float)/1e6
    
    total = nodata + notwater + perwater + seawater
    cloud = np.divide(nodata, total, out=np.zeros_like(nodata), where=total!=0)
    
    year = np.arange(1984, len(nodata)+1984)
    
    ##### use 0.2% for extremely large lakes #####
    ##### others use 5%
    if hylak_id in large_hylak_ids:
        mask = cloud <= 0.002
    else:
        mask = cloud <= 0.05
    
    ndf['year']         = np.repeat(year,    12)[2:]
    ndf['nodata']       = np.repeat(nodata,  12)[2:]
    ndf['notwater']     = np.repeat(notwater,12)[2:]
    ndf['perwater']     = np.repeat(perwater,12)[2:]
    ndf['seawater']     = np.repeat(seawater,12)[2:]
    ndf['cloud_pct_yr'] = np.repeat(cloud,   12)[2:]
    
    if np.sum(mask) >= 5:
        perwater = np.interp(year, np.extract(mask, year), np.extract(mask, perwater))
        seawater = np.interp(year, np.extract(mask, year), np.extract(mask, seawater))
        
    ndf['perwater_new'] = np.repeat(perwater,12)[2:]
    ndf['seawater_new'] = np.repeat(seawater,12)[2:]
    
    ##### calculate the reconstructed monthly #####
    season_frac = season_data[hylak_id - 2]
    
    new_monthly = np.repeat(perwater, 12) + np.repeat(seawater, 12) * np.tile(season_frac, len(perwater))
    ndf['recons_mthly'] = new_monthly[2:]
    
    ndf_new = yr_constrain(ndf)
    
    ##########################################################################
    ndf_new['9constrain'] = [0 if c==d else 1 for c,d in 
                             zip(ndf_new['3water_enh'].values, ndf_new['8water_area'].values)]
    ndf_new = ndf_new.rename({'perwater_new':'10lower_bound', 'recons_mthly':'12recons_mthly'}, axis=1)
    ndf_new['11upper_bound'] = ndf_new['10lower_bound'] + ndf_new['seawater_new']
    ndf_new = ndf_new[['hylak_id','1month','2cloud_pct','3water_enh','4water_noenh',
                       '5water_enh_raw','6water_noenh_raw','7intp_flag','8water_area',
                       '9constrain', '10lower_bound', '11upper_bound', '12recons_mthly']]
    ndf_new[['2cloud_pct','3water_enh','4water_noenh', '5water_enh_raw','6water_noenh_raw',
             '7intp_flag','8water_area','9constrain', '10lower_bound', '11upper_bound', '12recons_mthly']] = \
    ndf_new[['2cloud_pct','3water_enh','4water_noenh', '5water_enh_raw','6water_noenh_raw',
             '7intp_flag','8water_area','9constrain', '10lower_bound', '11upper_bound', '12recons_mthly']].astype(np.float32)
    return ndf_new

# 3. loop dataframe

In [8]:
for area_num in range(0, 4, 1):
    
    with zipfile.ZipFile("0_csv_monthly/HydroLakes_areas" + str(area_num) + ".zip", "r") as zipf:
        csv_data = []
        k = 0
        for fname in zipf.namelist():
            #print(fname)
            if '.csv' in fname:
                with zipf.open(fname) as myZip:
                    csv_data.append(pd.read_csv(myZip))
                
                if len(csv_data) >= 1000:  ## 1000 files == 100000 rows
                    
                    all_data = []
                    mdata = pd.concat(csv_data)
                    mdata = mdata.merge(all_data_annual, on='Hylak_id', how='inner').drop(columns=['system:index','.geo']) 
                    for index,row in mdata.iterrows():
                        all_data.append(row2df(row))
                        if index%1000 == 0:
                            print(index, process.memory_info().rss/1e6, datetime.now())
                    
                    pd.concat(all_data).to_parquet("2_intp_monthly/monthly_area_" + str(area_num) + 
                                                   '_' + str(k) + ".parquet", 
                                    index=False, compression='gzip', engine='pyarrow')
                    k = k + 1
                    csv_data = []
                        
        if len(csv_data) > 0:
            
            all_data = []
            mdata = pd.concat(csv_data)
            mdata = mdata.merge(all_data_annual, on='Hylak_id', how='inner').drop(columns=['system:index','.geo']) 
            for index,row in mdata.iterrows():
                all_data.append(row2df(row))
                if index%1000 == 0:
                    print(index, process.memory_info().rss/1e6, datetime.now())
            pd.concat(all_data).to_parquet("2_intp_monthly/monthly_area_" + str(area_num) + 
                                           '_' + str(k) + ".parquet", 
                            index=False, compression='gzip', engine='pyarrow')

0 5647.343616 2021-12-07 22:36:09.293835
1000 5661.585408 2021-12-07 22:36:39.648057
2000 5705.166848 2021-12-07 22:37:10.489253
3000 5749.260288 2021-12-07 22:37:41.311058
4000 5792.833536 2021-12-07 22:38:11.926037
5000 5836.73856 2021-12-07 22:38:42.542199
6000 5880.36096 2021-12-07 22:39:13.051722
7000 5924.43392 2021-12-07 22:39:43.763576
8000 5968.211968 2021-12-07 22:40:14.635594
9000 6011.990016 2021-12-07 22:40:45.258360
10000 6056.194048 2021-12-07 22:41:16.070589
11000 6099.820544 2021-12-07 22:41:47.089963
12000 6143.717376 2021-12-07 22:42:17.908228
13000 6187.343872 2021-12-07 22:42:48.750827
14000 6231.281664 2021-12-07 22:43:19.475365
15000 6275.1744 2021-12-07 22:43:50.000366
16000 6318.772224 2021-12-07 22:44:20.306509
17000 6362.570752 2021-12-07 22:44:51.229421
18000 6406.586368 2021-12-07 22:45:22.078107
19000 6450.42176 2021-12-07 22:45:52.481671
20000 6494.511104 2021-12-07 22:46:22.825176
21000 6538.104832 2021-12-07 22:46:53.362413
22000 6582.00576 2021-12-07 2

82000 11239.817216 2021-12-08 00:08:38.131751
83000 11249.819648 2021-12-08 00:09:09.492170
84000 11259.551744 2021-12-08 00:09:40.013314
85000 11269.824512 2021-12-08 00:10:10.699226
86000 11279.826944 2021-12-08 00:10:41.196917
87000 11289.829376 2021-12-08 00:11:11.746533
88000 11299.831808 2021-12-08 00:11:42.557648
89000 11309.83424 2021-12-08 00:12:12.935723
90000 11319.836672 2021-12-08 00:12:43.405825
91000 11329.839104 2021-12-08 00:13:14.024765
92000 11339.841536 2021-12-08 00:13:44.519904
93000 11349.843968 2021-12-08 00:14:14.710695
94000 11359.576064 2021-12-08 00:14:44.836701
95000 11369.578496 2021-12-08 00:15:14.922229
96000 11379.580928 2021-12-08 00:15:44.933374
97000 11389.58336 2021-12-08 00:16:15.161465
98000 11399.585792 2021-12-08 00:16:45.549514
99000 11409.588224 2021-12-08 00:17:15.967803
0 10570.481664 2021-12-08 00:19:04.582092
1000 10570.481664 2021-12-08 00:19:33.486367
2000 10570.481664 2021-12-08 00:20:02.522830
3000 10570.481664 2021-12-08 00:20:31.7229

62000 11023.773696 2021-12-08 01:39:08.352828
63000 11033.505792 2021-12-08 01:39:36.764989
64000 11043.508224 2021-12-08 01:40:05.787373
65000 11053.24032 2021-12-08 01:40:34.467354
66000 11062.70208 2021-12-08 01:41:01.730541
67000 11072.16384 2021-12-08 01:41:29.317243
68000 11081.895936 2021-12-08 01:41:57.780569
69000 11091.628032 2021-12-08 01:42:26.235966
70000 11101.360128 2021-12-08 01:42:55.791215
71000 11111.36256 2021-12-08 01:43:24.274179
72000 11121.364992 2021-12-08 01:43:52.769298
73000 11131.097088 2021-12-08 01:44:21.281395
74000 11141.09952 2021-12-08 01:44:49.914413
75000 11150.831616 2021-12-08 01:45:18.744503
76000 11160.834048 2021-12-08 01:45:47.551672
77000 11170.566144 2021-12-08 01:46:16.161041
78000 11180.568576 2021-12-08 01:46:45.199270
79000 11190.571008 2021-12-08 01:47:13.937579
80000 11200.303104 2021-12-08 01:47:42.004741
81000 11210.0352 2021-12-08 01:48:10.480514
82000 11219.767296 2021-12-08 01:48:39.039134
83000 11229.769728 2021-12-08 01:49:07.92

42000 10824.605696 2021-12-08 03:09:46.797187
43000 10834.608128 2021-12-08 03:10:17.386759
44000 10844.340224 2021-12-08 03:10:48.368887
45000 10854.342656 2021-12-08 03:11:18.850385
46000 10864.615424 2021-12-08 03:11:49.136596
47000 10874.617856 2021-12-08 03:12:19.613129
48000 10884.620288 2021-12-08 03:12:49.308507
49000 10894.62272 2021-12-08 03:13:19.094223
50000 10893.139968 2021-12-08 03:13:49.255156
51000 10903.1424 2021-12-08 03:14:19.011705
52000 10913.144832 2021-12-08 03:14:48.749752
53000 10923.147264 2021-12-08 03:15:18.495297
54000 10932.87936 2021-12-08 03:15:48.841167
55000 10943.152128 2021-12-08 03:16:18.553393
56000 10953.15456 2021-12-08 03:16:48.338454
57000 10963.156992 2021-12-08 03:17:18.077885
58000 10973.159424 2021-12-08 03:17:47.897449
59000 10983.161856 2021-12-08 03:18:17.637059
60000 10993.164288 2021-12-08 03:18:47.355769
61000 11003.16672 2021-12-08 03:19:17.177023
62000 11013.169152 2021-12-08 03:19:46.886306
63000 11023.171584 2021-12-08 03:20:16.6

22000 10683.027456 2021-12-08 04:39:56.819096
23000 10683.027456 2021-12-08 04:40:26.582235
24000 10683.027456 2021-12-08 04:40:56.126947
25000 10683.027456 2021-12-08 04:41:25.691012
26000 10683.027456 2021-12-08 04:41:55.244406
27000 10683.027456 2021-12-08 04:42:24.750865
28000 10683.027456 2021-12-08 04:42:54.561972
29000 10683.027456 2021-12-08 04:43:24.133584
30000 10688.43008 2021-12-08 04:43:53.583749
31000 10698.432512 2021-12-08 04:44:23.166255
32000 10708.434944 2021-12-08 04:44:52.686632
33000 10718.437376 2021-12-08 04:45:22.189262
34000 10728.439808 2021-12-08 04:45:51.562769
35000 10738.171904 2021-12-08 04:46:21.427444
36000 10748.444672 2021-12-08 04:46:50.882844
37000 10758.447104 2021-12-08 04:47:20.275177
38000 10768.449536 2021-12-08 04:47:49.599572
39000 10778.451968 2021-12-08 04:48:18.943374
40000 10788.4544 2021-12-08 04:48:48.295146
41000 10798.456832 2021-12-08 04:49:17.663695
42000 10808.188928 2021-12-08 04:49:47.479153
43000 10818.461696 2021-12-08 04:50:1

2000 10715.652096 2021-12-08 06:10:13.482266
3000 10715.652096 2021-12-08 06:10:42.657765
4000 10715.652096 2021-12-08 06:11:11.894427
5000 10715.652096 2021-12-08 06:11:41.033696
6000 10715.652096 2021-12-08 06:12:10.309419
7000 10715.652096 2021-12-08 06:12:39.587102
8000 10715.652096 2021-12-08 06:13:09.291418
9000 10715.652096 2021-12-08 06:13:39.716235
10000 10715.652096 2021-12-08 06:14:09.361424
11000 10715.652096 2021-12-08 06:14:38.730194
12000 10715.652096 2021-12-08 06:15:08.037097
13000 10715.652096 2021-12-08 06:15:37.487554
14000 10715.652096 2021-12-08 06:16:07.156251
15000 10715.652096 2021-12-08 06:16:36.673934
16000 10715.652096 2021-12-08 06:17:06.454208
17000 10715.652096 2021-12-08 06:17:35.744791
18000 10715.652096 2021-12-08 06:18:05.277269
19000 10715.652096 2021-12-08 06:18:34.871772
20000 10715.652096 2021-12-08 06:19:04.560710
21000 10715.488256 2021-12-08 06:19:34.298986
22000 10715.488256 2021-12-08 06:20:03.153452
23000 10715.488256 2021-12-08 06:20:32.255

82000 11168.202752 2021-12-08 07:40:35.522449
83000 11178.205184 2021-12-08 07:41:05.421809
84000 11188.207616 2021-12-08 07:41:35.330391
85000 11197.939712 2021-12-08 07:42:06.113139
86000 11208.21248 2021-12-08 07:42:36.064458
87000 11218.214912 2021-12-08 07:43:06.052992
88000 11228.217344 2021-12-08 07:43:35.517770
89000 11238.219776 2021-12-08 07:44:04.906121
90000 11248.222208 2021-12-08 07:44:34.558739
91000 11258.22464 2021-12-08 07:45:04.308550
92000 11268.227072 2021-12-08 07:45:33.861638
93000 11278.229504 2021-12-08 07:46:03.390154
94000 11288.231936 2021-12-08 07:46:33.103665
95000 11298.234368 2021-12-08 07:47:03.025111
96000 11308.2368 2021-12-08 07:47:32.836930
97000 11318.239232 2021-12-08 07:48:02.729882
98000 11328.241664 2021-12-08 07:48:32.460626
99000 11338.244096 2021-12-08 07:49:02.570723
0 10810.392576 2021-12-08 07:50:49.329659
1000 10797.584384 2021-12-08 07:51:19.384988
2000 10797.584384 2021-12-08 07:51:48.919108
3000 10797.584384 2021-12-08 07:52:18.930492

62000 11012.620288 2021-12-08 09:08:44.081131
63000 11022.352384 2021-12-08 09:09:12.066497
64000 11031.814144 2021-12-08 09:09:40.059002
65000 11041.54624 2021-12-08 09:10:06.593064
66000 11050.196992 2021-12-08 09:10:30.682679
67000 11058.577408 2021-12-08 09:10:54.590416
68000 11068.039168 2021-12-08 09:11:20.696192
69000 11077.500928 2021-12-08 09:11:47.111590
70000 11086.962688 2021-12-08 09:12:14.056604
71000 11096.424448 2021-12-08 09:12:40.851700
72000 11105.886208 2021-12-08 09:13:07.313993
73000 11115.618304 2021-12-08 09:13:34.541818
74000 11125.3504 2021-12-08 09:14:01.221528
75000 11134.81216 2021-12-08 09:14:28.056825
76000 11144.544256 2021-12-08 09:14:55.571451
77000 11154.276352 2021-12-08 09:15:23.013929
78000 11163.738112 2021-12-08 09:15:50.931616
79000 11173.740544 2021-12-08 09:16:18.193962
80000 11183.47264 2021-12-08 09:16:45.459481
81000 11193.475072 2021-12-08 09:17:13.164873
82000 11203.207168 2021-12-08 09:17:40.478686
83000 11212.939264 2021-12-08 09:18:07.

# 4. only area to npy

In [3]:
output = []
for fname in sorted(glob.glob('2_intp_monthly/monthly_area*.parquet')):
    print(fname, process.memory_info().rss/1e6, datetime.now())
    mdata = pd.read_parquet(fname, columns=['hylak_id','1month','8water_area'])
    mdata = mdata.pivot(index='hylak_id', columns='1month', values='8water_area').reset_index().values
    
    output.append(np.float32(mdata))

output_np = np.concatenate(output)
print(output_np)

2_intp_monthly/monthly_area_0_0.parquet 185.872384 2021-12-23 06:05:10.230211
2_intp_monthly/monthly_area_0_1.parquet 1621.573632 2021-12-23 06:05:20.250308
2_intp_monthly/monthly_area_0_2.parquet 1800.024064 2021-12-23 06:05:29.776127
2_intp_monthly/monthly_area_0_3.parquet 1965.576192 2021-12-23 06:05:40.099947
2_intp_monthly/monthly_area_0_4.parquet 2115.846144 2021-12-23 06:05:50.219152
2_intp_monthly/monthly_area_1_0.parquet 2293.8624 2021-12-23 06:06:00.575513
2_intp_monthly/monthly_area_1_1.parquet 2499.60448 2021-12-23 06:06:10.408760
2_intp_monthly/monthly_area_1_2.parquet 2647.687168 2021-12-23 06:06:20.384393
2_intp_monthly/monthly_area_1_3.parquet 2851.684352 2021-12-23 06:06:30.203442
2_intp_monthly/monthly_area_1_4.parquet 3029.045248 2021-12-23 06:06:39.705027
2_intp_monthly/monthly_area_2_0.parquet 3204.194304 2021-12-23 06:06:48.733024
2_intp_monthly/monthly_area_2_1.parquet 3383.037952 2021-12-23 06:06:58.361561
2_intp_monthly/monthly_area_2_2.parquet 3553.107968 2021

In [4]:
real_monthly_ids = output_np[:,0].astype(int)
all_ids = np.arange(1,1427689,1)

insert_ids = np.sort(list(set(all_ids) - set(real_monthly_ids)))
np.savetxt('2_intp_monthly/out_monthly_combine_usedPseudo.csv', insert_ids, fmt='%d', delimiter='\n')
print(insert_ids)

[      1       2       3 ... 1427686 1427687 1427688]


In [5]:
pseudo_monthly = np.load('1_intp_annual/pseudo_monthly.npy')[:,2:]  ## from Mar 1984
print(pseudo_monthly.shape)

(1427688, 442)


In [6]:
print(output_np.shape)

(1412596, 443)


In [7]:
for i in range(len(pseudo_monthly)):
    hylak_id = i + 1
    real_index = np.where(real_monthly_ids == hylak_id)[0]
    if len(real_index) == 1:
        pseudo_monthly[i] = output_np[real_index[0]][1:]     ## remove hylak_id
    else:
        pseudo_monthly[i] = pseudo_monthly[i]/1e6            ## m2 to km2
    if i%10000 == 0:
        print(i, process.memory_info().rss/1e6, datetime.now())
    
np.save('2_intp_monthly/out_monthly_combine.npy', np.float32(pseudo_monthly))

0 7808.094208 2021-12-23 06:10:07.432133
10000 7808.094208 2021-12-23 06:10:15.784358
20000 7808.094208 2021-12-23 06:10:24.058423
30000 7808.094208 2021-12-23 06:10:32.413321
40000 7808.094208 2021-12-23 06:10:40.767884
50000 7808.094208 2021-12-23 06:10:49.123588
60000 7808.094208 2021-12-23 06:10:57.475037
70000 7808.094208 2021-12-23 06:11:05.849403
80000 7808.094208 2021-12-23 06:11:14.198268
90000 7808.094208 2021-12-23 06:11:22.557057
100000 7808.094208 2021-12-23 06:11:30.906483
110000 7808.094208 2021-12-23 06:11:39.257455
120000 7808.094208 2021-12-23 06:11:47.577753
130000 7808.094208 2021-12-23 06:11:55.889136
140000 7808.094208 2021-12-23 06:12:04.236592
150000 7808.094208 2021-12-23 06:12:12.571756
160000 7808.094208 2021-12-23 06:12:20.880841
170000 7808.094208 2021-12-23 06:12:29.217191
180000 7808.094208 2021-12-23 06:12:37.551935
190000 7808.094208 2021-12-23 06:12:45.550161
200000 7808.094208 2021-12-23 06:12:53.899761
210000 7808.094208 2021-12-23 06:13:02.247261
22